# NLLB

In [ ]:
!pip install -q transformers torch accelerate

In [ ]:
import torch
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

def load_nllb_pipeline(model_id="facebook/nllb-200-distilled-600M"):
    # GPU 가용 여부 확인
    device = 0 if torch.cuda.is_available() else -1

    print(f"Loading model: {model_id} on {'GPU' if device == 0 else 'CPU'}...")

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

    # 번역 파이프라인 구축
    # eng_Latn: 영어(라틴어 기반), kor_Hang: 한국어(한글 기반)
    translation_pipe = pipeline(
        "translation",
        model=model,
        tokenizer=tokenizer,
        src_lang="eng_Latn",
        tgt_lang="kor_Hang",
        device=device
    )
    return translation_pipe

# 1. 모델 로드
nllb_translator = load_nllb_pipeline()

# 2. 테스트용 영문 캡션 리스트 (VLM 결과물 예시)
eng_captions = [
    "A funny looking white dog running across a sunlit park.",
    "A close-up shot of a delicious pepperoni pizza with melting cheese.",
    "A futuristic city skyline at night with neon lights and flying cars."
]

# 3. 번역 실행
print("\n--- 번역 결과 ---")
for caption in eng_captions:
    # max_length는 캡션 길이에 따라 조절 가능
    result = nllb_translator(caption, max_length=128)
    ko_text = result[0]['translation_text']
    print(f"EN: {caption}")
    print(f"KO: {ko_text}\n")

In [ ]:
eng_captions = [
    "A cute white dog running across a sunlit park.",
    "A close-up shot of a delicious pepperoni pizza with melting cheese.",
    "A futuristic city skyline at night with neon lights and flying cars."
]

# 3. 번역 실행
print("\n--- 번역 결과 ---")
for caption in eng_captions:
    # max_length는 캡션 길이에 따라 조절 가능
    result = nllb_translator(caption, max_length=128)
    ko_text = result[0]['translation_text']
    print(f"EN: {caption}")
    print(f"KO: {ko_text}\n")

# EXAONE

In [ ]:
dr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q transformers accelerate bitsandbytes

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

# 1. 모델 및 토크나이저 설정
model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"

# 4비트 양자화 설정 (코랩 T4 GPU 메모리 확보용)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print("EXAONE 3.5 모델을 로드 중입니다... (시간이 다소 소요될 수 있습니다)")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# 2. 번역 함수 정의
def translate_to_korean(eng_text):
    # EXAONE 전용 프롬프트 스타일 (임베딩에 적합하게 명사 위주/간결한 번역 요청)
    messages = [
        {"role": "system", "content": "너는 유능한 번역가야. 입력되는 영어 텍스트를 임베딩 작업에 적합하도록 군더더기 없이 자연스러운 한국어로 번역해줘."},
        {"role": "user", "content": f"다음 문장을 한국어로 번역해: {eng_text}"}
    ]

    # 템플릿 적용
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # 추론
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False, # 결과의 일관성을 위해 False
            eos_token_id=tokenizer.eos_token_id
        )

    # 결과 텍스트만 추출
    result = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return result.strip()

# 3. 테스트 실행
eng_caption = "A golden retriever playing with a red ball on a green grass field."
ko_translation = translate_to_korean(eng_caption)

print(f"\n[VLM 결과]: {eng_caption}")
print(f"[EXAONE 번역]: {ko_translation}")

In [ ]:
# 3. 테스트 실행
eng_caption = "A golden retriever playing with a red ball on a green grass field."
ko_translation = translate_to_korean(eng_caption)

print(f"\n[VLM 결과]: {eng_caption}")
print(f"[EXAONE 번역]: {ko_translation}")

In [ ]:
import os
import json
import glob
from tqdm import tqdm # 진행률 표시용

# 1. 경로 설정
input_folder = "/content/drive/MyDrive/BOAZ_/split_jsons"
# 원본 보호를 위해 결과 전용 폴더를 만드는 것을 추천합니다.
output_folder = "/content/drive/MyDrive/BOAZ_/split_jsons_translated"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 2. EXAONE 번역 함수 (캡션 특화 최적화)
def translate_caption(eng_text):
    # 캡션은 키워드가 나열된 경우가 많으므로, 문맥을 살리되 간결하게 번역하도록 프롬프트 구성
    messages = [
        {"role": "system", "content": "너는 웹툰 캡션 전문 번역가야. 입력되는 영어 묘사들을 자연스럽고 핵심 위주의 한국어로 번역해줘. 설명 없이 번역 결과만 출력해."},
        {"role": "user", "content": f"번역해: {eng_text}"}
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512, # 캡션이 길 수 있으므로 충분히 설정
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id
        )

    result = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return result.strip()

# 3. 폴더 내 파일 리스트 가져오기
json_files = glob.glob(os.path.join(input_folder, "*.json"))
print(f"총 {len(json_files)}개의 파일을 찾았습니다.")

# 4. 루프를 돌며 번역 실행
for file_path in tqdm(json_files, desc="번역 진행 중"):
    try:
        # 파일 읽기
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # 'caption' 키가 있는지 확인 후 번역
        if "caption" in data:
            english_caption = data["caption"]
            korean_caption = translate_caption(english_caption)

            # 데이터 업데이트
            data["caption"] = korean_caption

            # 새 파일명 및 저장 (파일명은 그대로 유지)
            file_name = os.path.basename(file_path)
            save_path = os.path.join(output_folder, file_name)

            with open(save_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
        else:
            print(f"\n[Skip] 'caption' 키가 없습니다: {file_path}")

    except Exception as e:
        print(f"\n[Error] {file_path} 처리 중 오류 발생: {e}")

print(f"\n모든 작업이 완료되었습니다! 결과는 '{output_folder}'에서 확인하세요.")